In [69]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [70]:
dots = [(5,2), (1,2), (2,1), (6,2),
        (1,1), (3,1), (7,-1), (5,-1)]
print(dots)

[(5, 2), (1, 2), (2, 1), (6, 2), (1, 1), (3, 1), (7, -1), (5, -1)]


In [71]:
K = 2

In [72]:
def calDunn(d:dict):
    ## 分子
    mindis = float('inf')
    for k1, v1 in d.items():
        for k2, v2 in d.items():
            if k1 < k2:
                for i in v1:
                    for j in v2:
                        dis = np.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2)
                        if dis < mindis:
                            mindis = dis
    ## 分母
    maxdis = 0
    for k, v in d.items():
        for i in v:
            for j in v:
                dis = np.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2)
                if dis > maxdis:
                    maxdis = dis
    return mindis/maxdis

In [73]:
def calDB(d:dict):
    keys = list(d.keys())
    classnum = len(keys)
    sum = 0
    for k1, _ in d.items():
        temp = 0
        for k2, _ in d.items():
            if k1 != k2:
                temp = max(temp, calRij(d, k1, k2))
        sum += temp
    return sum/classnum
            
def calRij(d:dict, i, j):
    m1 = (np.mean([x for x, _ in d[i]]), np.mean([y for _, y in d[i]]))
    m2 = (np.mean([x for x, _ in d[j]]), np.mean([y for _, y in d[j]]))
    dij = np.sqrt((m1[0]-m2[0])**2 + (m1[1]-m2[1])**2)
    si = np.sqrt(
        sum([(x-m1[0])**2 + (y-m1[1])**2 for x, y in d[i]])
        /len(d[i])
        )
    sj = np.sqrt(
        sum([(x-m2[0])**2 + (y-m2[1])**2 for x, y in d[j]])
        /len(d[j])
        )
    return (si+sj)/dij

### 距离的度量
- 一个样本A与一个类C的距离：用样本A与类C的样本均值之间的距离来标识
- 类C1与类C2的距离：用类C1的样本均值与类C2的样本均值的距离来标识

### 顺序聚类

In [74]:
class SequenticalCluster():
    def __init__(self, dots, K):
        self.dots = dots
        self.K = K
        self.centers = dict()
        self.cnt = 0
        self.result = dict()
        for i in range(K):
            self.result[i] = []
    
    def fit(self):
        for dot in self.dots:
            print("******输入的点:", dot, '******')
            if self.cnt < K:
                self.centers[self.cnt] = dot
                self.result[self.cnt].append(dot)
                self.cnt += 1
                print("初始化中心点:", self.centers)
                continue
            index = self.__findnearest(dot)
            self.result[index].append(dot)
            print("聚类结果",self.result)
            self.__updatecenter()
        return self.result
    
    def __findnearest(self, dot):
        min_dis = float('inf')
        min_index = -1
        for i in range(K):
            dis = np.sqrt((self.centers[i][0]-dot[0])**2 + (self.centers[i][1]-dot[1])**2)
            print(f"到类别{i}中心点的距离为{dis}")
            if dis < min_dis:
                min_dis = dis
                min_index = i
        return min_index
    
    def __updatecenter(self):
        for i in range(K):
            sumx = [x for x, _ in self.result[i]]
            sumy = [y for _, y in self.result[i]]
            self.centers[i] = (sum(sumx)/len(sumx), sum(sumy)/len(sumy))
        print('更新中心点', self.centers)
            

sc = SequenticalCluster(dots, K)
res = sc.fit()
print("聚类结果:", res)
print("Dunn指数:", calDunn(res))
print("Davies-Bouldin指数:", calDB(res))

******输入的点: (5, 2) ******
初始化中心点: {0: (5, 2)}
******输入的点: (1, 2) ******
初始化中心点: {0: (5, 2), 1: (1, 2)}
******输入的点: (2, 1) ******
到类别0中心点的距离为3.1622776601683795
到类别1中心点的距离为1.4142135623730951
聚类结果 {0: [(5, 2)], 1: [(1, 2), (2, 1)]}
更新中心点 {0: (5.0, 2.0), 1: (1.5, 1.5)}
******输入的点: (6, 2) ******
到类别0中心点的距离为1.0
到类别1中心点的距离为4.527692569068709
聚类结果 {0: [(5, 2), (6, 2)], 1: [(1, 2), (2, 1)]}
更新中心点 {0: (5.5, 2.0), 1: (1.5, 1.5)}
******输入的点: (1, 1) ******
到类别0中心点的距离为4.6097722286464435
到类别1中心点的距离为0.7071067811865476
聚类结果 {0: [(5, 2), (6, 2)], 1: [(1, 2), (2, 1), (1, 1)]}
更新中心点 {0: (5.5, 2.0), 1: (1.3333333333333333, 1.3333333333333333)}
******输入的点: (3, 1) ******
到类别0中心点的距离为2.692582403567252
到类别1中心点的距离为1.699673171197595
聚类结果 {0: [(5, 2), (6, 2)], 1: [(1, 2), (2, 1), (1, 1), (3, 1)]}
更新中心点 {0: (5.5, 2.0), 1: (1.75, 1.25)}
******输入的点: (7, -1) ******
到类别0中心点的距离为3.3541019662496847
到类别1中心点的距离为5.7118298293979315
聚类结果 {0: [(5, 2), (6, 2), (7, -1)], 1: [(1, 2), (2, 1), (1, 1), (3, 1)]}
更新中心点 {0: (6.0, 1.0), 1

### 谱系聚类

In [75]:
class HierarchicalCluster():
    def __init__(self, dots, K):
        self.dots = dots
        self.K = K
        self.nowclassnum = len(dots)
        self.dictls:dict[list] = dict()
        for i in range(0, self.nowclassnum):
            self.dictls[i] = [dots[i]]
        print("初始化")
        print(self.dictls)
    
    def fit(self):
        while self.nowclassnum > self.K:
            self.__merge()
            self.nowclassnum -= 1
        return self.dictls
    
    def __merge(self):
        min_dis = float('inf')
        midnex0 = -1
        midnex1 = -1
        for k1, v1 in self.dictls.items():
            for k2, v2 in self.dictls.items():
                if k1 < k2:
                    dis = self.__getdis(v1, v2)
                    if dis < min_dis:
                        min_dis = dis
                        midnex0 = k1
                        midnex1 = k2
        if midnex0 == -1 or midnex1 == -1:
            raise ValueError("合并失败")
        print(f"合并类{midnex0}和类{midnex1}为类{midnex0}")
        self.dictls[midnex0] = self.dictls[midnex0] + self.dictls[midnex1]
        del self.dictls[midnex1]
        print(self.dictls)
    
    def __getdis(self, v1, v2):
        sum = 0
        v1center = (np.mean([x for x, _ in v1]), np.mean([y for _, y in v1]))
        v2center = (np.mean([x for x, _ in v2]), np.mean([y for _, y in v2]))
        return ((v1center[0]-v2center[0])**2 + (v1center[1]-v2center[1])**2)**0.5

hc = HierarchicalCluster(dots, K)
res = hc.fit()
print("聚类结果:", res)
print("Dunn指数:", calDunn(res))
print("Davies-Bouldin指数:", calDB(res))

初始化
{0: [(5, 2)], 1: [(1, 2)], 2: [(2, 1)], 3: [(6, 2)], 4: [(1, 1)], 5: [(3, 1)], 6: [(7, -1)], 7: [(5, -1)]}
合并类0和类3为类0
{0: [(5, 2), (6, 2)], 1: [(1, 2)], 2: [(2, 1)], 4: [(1, 1)], 5: [(3, 1)], 6: [(7, -1)], 7: [(5, -1)]}
合并类1和类4为类1
{0: [(5, 2), (6, 2)], 1: [(1, 2), (1, 1)], 2: [(2, 1)], 5: [(3, 1)], 6: [(7, -1)], 7: [(5, -1)]}
合并类2和类5为类2
{0: [(5, 2), (6, 2)], 1: [(1, 2), (1, 1)], 2: [(2, 1), (3, 1)], 6: [(7, -1)], 7: [(5, -1)]}
合并类1和类2为类1
{0: [(5, 2), (6, 2)], 1: [(1, 2), (1, 1), (2, 1), (3, 1)], 6: [(7, -1)], 7: [(5, -1)]}
合并类6和类7为类6
{0: [(5, 2), (6, 2)], 1: [(1, 2), (1, 1), (2, 1), (3, 1)], 6: [(7, -1), (5, -1)]}
合并类0和类6为类0
{0: [(5, 2), (6, 2), (7, -1), (5, -1)], 1: [(1, 2), (1, 1), (2, 1), (3, 1)]}
聚类结果: {0: [(5, 2), (6, 2), (7, -1), (5, -1)], 1: [(1, 2), (1, 1), (2, 1), (3, 1)]}
Dunn指数: 0.6201736729460423
Davies-Bouldin指数: 0.6509877005287489


### K均值聚类


In [76]:
class KMeans():
    eps = 1e-6
    def __init__(self, k, dots):
        self.k = k
        self.dots = dots
        self.kcenterdict = dict()
        self.kdots = dict()
        self.state = random.getstate()
        self.__initrandomcenter()
        self.__initkdots()
        self.changed = True
        self.internum = 0
        
    
    def __initrandomcenter(self):
        random.setstate(self.state)
        temp = random.sample(range(0, len(self.dots)), self.k)
        for i in range(self.k):
            self.kcenterdict[i] = self.dots[temp[i]]
        print("初始随机中心点\n", self.kcenterdict)
        
    def __initkdots(self):
        for i in range(self.k):
            self.kdots[i] = []
            
    def fit(self):
        while self.changed:
            print("迭代次数", self.internum)
            print("类中心点", self.kcenterdict)
            self.__initkdots()
            for dot in self.dots:
                nearest = self.__getnearest(dot)
                self.kdots[nearest].append(dot)
            self.__updatecenter()
            self.internum += 1
        return self.kdots, self.kcenterdict
    
    def __updatecenter(self):
        for i in range(self.k):
            newx = sum([x for x, _ in self.kdots[i]]) / len(self.kdots[i])
            newy = sum([y for _, y in self.kdots[i]]) / len(self.kdots[i])
            if(abs(newx - self.kcenterdict[i][0]) < self.eps and abs(newy - self.kcenterdict[i][1]) < self.eps):
                self.changed = False
            else:
                # self.changed = True
                self.kcenterdict[i] = (newx, newy)

    def __getnearest(self, dot):
        mindis = float('inf') 
        index = None
        for i in range(self.k):
            x1, y1 = dot
            x2, y2 = self.kcenterdict[i]
            dis = ((x1 - x2) ** 2 + (y1 - y2) ** 2)**0.5
            if dis < mindis:
                mindis = dis
                index = i
        if index is None:
            raise ValueError("index is None")
        return index

km = KMeans(K, dots)
res, kcenter = km.fit()
print("聚类结果", res)
print("聚类中心点", kcenter)
print("Dunn指数:", calDunn(res))
print("Davies-Bouldin指数:", calDB(res))

初始随机中心点
 {0: (7, -1), 1: (5, -1)}
迭代次数 0
类中心点 {0: (7, -1), 1: (5, -1)}
迭代次数 1
类中心点 {0: (6.5, 0.5), 1: (2.8333333333333335, 1.0)}
迭代次数 2
类中心点 {0: (5.75, 0.5), 1: (1.75, 1.25)}
聚类结果 {0: [(5, 2), (6, 2), (7, -1), (5, -1)], 1: [(1, 2), (2, 1), (1, 1), (3, 1)]}
聚类中心点 {0: (5.75, 0.5), 1: (1.75, 1.25)}
Dunn指数: 0.6201736729460423
Davies-Bouldin指数: 0.6509877005287489
